In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
translation_dict = {
    'native-tls-hw-supp': 'Native TLS HW supp',
    'native-tls': 'Native TLS no HW supp',
    'native': 'Native (no TLS)',
    'wasm': 'WASM (no TLS)',
    'wasm-tls': 'WASM TLS',
    'sgx': 'WASM SGX (no TLS)',
    'sgx-tls': 'WASM SGX TLS',
    'qos0': 'QoS 0',
    'qos1': 'QoS 1',
    'qos2': 'QoS 2',
    'expected_delivered': 'Expected delivered messages / s',
    'delivered_messages': 'Delivered messages / s',
    'reliability': 'Delivery rate (%)',
    'latency': 'Latency (s)',
    'throughput': 'Throughput (Messages /s) [at min 95 % reliability]',
    'timestamp' : 'Time [s]',
    'messages' : 'Messages / s',
}

cases = ['native-tls', 'native-tls-hw-supp', 'native', 'wasm', 'wasm-tls', 'sgx', 'sgx-tls']
cases_tls = ['native-tls', 'wasm-tls', 'sgx-tls']
cases_no_tls = ['native', 'wasm', 'sgx']
cases_native = ['native-tls', 'native-tls-hw-supp', 'native']

cases = cases_tls

machine = 'eiger-1'
machine = 'grassen-1'

In [ ]:
number_publishers = 1
number_subscribers = 1
message_dict = {}
for case in cases:
    publisher_df = pd.DataFrame()
    for i in range(number_publishers):
        df = pd.read_csv('data/' + machine + '/results_' + case + '/' + str(i) + '.csv', sep = ';')
        publisher_df = pd.concat([publisher_df, df])
    
    subscriber_df = pd.DataFrame()
    for i in range(number_subscribers):
        df = pd.read_csv('data/' + machine + '/results_' + case + '/' + str(i + number_publishers) + '.csv', sep = ';')
        subscriber_df = pd.concat([subscriber_df, df])
    
    # only receive events
    subscriber_df = subscriber_df[subscriber_df['event'] == 9]
    subscriber_df = subscriber_df.sort_values(by=['timestamp']).rename(columns={'timestamp': 'timestamp_receive'})
    subscriber_df = subscriber_df.drop(columns=['event'])
    # only send events
    publisher_df = publisher_df[publisher_df['event'] == 3]
    publisher_df = publisher_df.sort_values(by=['timestamp']).rename(columns={'timestamp': 'timestamp_send'})
    publisher_df = publisher_df.drop(columns=['event'])

    print('Case: ' + case)
    print('Number of received messages: ' + str(len(subscriber_df)))
    print('Number of sent messages: ' + str(len(publisher_df)))
    print('Number of lost messages: ' + str(len(publisher_df) - len(subscriber_df)))
    print('\n')

    df = pd.merge(subscriber_df, publisher_df, on='payload', how='outer')
    df = df.drop(columns=['payload'])

    df['latency'] = df['timestamp_receive'] - df['timestamp_send']
    
    message_dict[case] = df


In [ ]:
# if directory export does not exist, create it
if not os.path.exists('export'):
    os.makedirs('export')
if not os.path.exists('export/' + machine):
    os.makedirs('export/' + machine)

f = open('export/' + machine + '/readme.txt', 'w')
f.write('This folder contains the results of the case "message rate scaling".\n')
f.write('In this case, the number of subscribers and publishers is fixed.\n')
f.write('The message rate is per second and scales at an irregular rate: 5, 25, 50, 100, 200.\n')
f.write('Every case runs for 60 seconds. Afterwards the number is increased. Note, that the broker is not restarted inbetween.\n\n')
f.write('The parameters used are as follows:\n')
f.write('  - Number of subscribers: ' + str(number_subscribers) + '\n')
f.write('  - Number of publishers: ' + str(number_publishers) + '\n')
f.write('  - Size of Payload: 16Kb random data\n')
f.write('  - QoS: 0\n')
f.write('\n')
f.write('We plot the average latency per message rate.')
f.close()

In [ ]:
bin_dict = {}
for case in cases:
    df = pd.read_csv('data/' + machine + '/results_' + case + '/orchestrator.csv', sep = ';')
    # iterate through df
    bins = pd.DataFrame(columns = ['bin', 'start', 'end', 'duration'])
    for index, row in df.iterrows():
        # add new bin
        bins = bins.append({'bin': row['payload'], 'start': row['timestamp']}, ignore_index = True)

        # check if bin is already in bins
        if index > 0:
            # update end time
            bins.iloc[index-1]['end'] = row['timestamp']
            # update duration
            bins.iloc[index-1]['duration'] = bins.iloc[index-1]['end'] - bins.iloc[index-1]['start'] 
        
        if index == len(df) - 1:
            bins.iloc[index]['end'] = message_dict[case]['timestamp_receive'].max()
            bins.iloc[index]['duration'] = bins.iloc[index]['end'] - bins.iloc[index]['start'] 
        
    bin_dict[case] = bins

In [ ]:
# assign every message to one of the bins
for case in cases:
    result_df = pd.DataFrame()
    message_df = message_dict[case]
    message_df['message_rate'] = 0
    bin_df = bin_dict[case]
    for index, row in bin_df.iterrows():
        # get all messages which have send timestamp between start and end
        start = row['start']
        end = row['end']
        message_df.loc[(message_df['timestamp_send'] >= start) & (message_df['timestamp_send'] < end), 'message_rate'] = row['bin']

    message_df = message_df[message_df['message_rate'] != 0]
    message_dict[case] = message_df

In [ ]:
message_latency = pd.DataFrame()
for case in cases:
    df = message_dict[case]
    df['case'] = case
    message_latency = pd.concat([message_latency, df])

message_latency.to_csv('export/' + machine + '/message_latency.csv')

# write a description file for the export
f = open('export/' + machine + '/message_latency.txt', 'w')
f.write('This file contains for every message the observed maximum latency.\n')
f.write('The first column is a index without any further meaning\n')
f.write('the other columns are as described below:\n\n')
for col in message_latency.columns:
    if(col == 'case'):
        f.write(col + ': case of the measurement\n')
    if(col == 'latency'):
        f.write(col + ': end to end latency for this given message in ms\n')
    if(col == 'message_rate'):
        f.write(col + ': the message rate of the publisher in place per second\n')
    if(col == 'timestamp_send'):
        f.write(col + ': the timestamp of when the publisher sent the message\n')
    if(col == 'timestamp_receive'):
        f.write(col + ': the timestamp of when the subscriber received the message\n')

f.write('\n\n')
f.write('The following cases were measured:\n')
for case in cases:
    f.write(case + ': ' + translation_dict[case] +'\n')

f.close()

In [ ]:
latency_per_rate = pd.DataFrame()
for case in cases:
    messages = message_dict[case]
    df = messages.groupby('message_rate').mean()
    df = df.drop(columns=['timestamp_send', 'timestamp_receive'])
    df['case'] = case
    df = df.reset_index()
    latency_per_rate = pd.concat([latency_per_rate, df])

In [ ]:
# if directory export does not exist, create it
if not os.path.exists('export'):
    os.makedirs('export')
if not os.path.exists('export/' + machine):
    os.makedirs('export/' + machine)
latency_per_rate.to_csv('export/' + machine + '/latency_per_rate.csv')

# write a description file for the export
f = open('export/' + machine + '/latency_per_rate.txt', 'w')
f.write('This file contains the average latency per message rate and case.\n')
f.write('The first column is a index without any further meaning\n')
f.write('the other columns are as described below:\n\n')
for col in latency_per_rate.columns:
    if(col == 'case'):
        f.write(col + ': case of the measurement\n')
    if(col == 'latency'):
        f.write(col + ': mean latency of all single message latencies\n')
    if(col == 'message_rate'):
        f.write(col + ': the message rate of the publisher in place per second\n')

f.write('\n\n')
f.write('The following cases were measured:\n')
for case in cases:
    f.write(case + ': ' + translation_dict[case] +'\n')

f.close()

In [ ]:
df = latency_per_rate.rename(columns={'latency': 'Latency (ms)', 'message_rate': 'message rate / s'})
sns.barplot(data=df, x='message rate / s', y='Latency (ms)', hue='case')
plt.title('Average latency by message rate / s\n['+ machine + ']')
plt.xlabel('message rate / s')
plt.ylabel('Latency (ms)')

plt.savefig('export/' + machine + '/latency_per_message_rate.png', dpi=300, bbox_inches='tight', transparent = False)

In [ ]:
sns.boxplot(data=message_latency, x='message_rate', y='latency', hue='case')
plt.title('Latency by message rate / s\n['+ machine + ']')
plt.xlabel('message rate / s')
plt.ylabel('Latency (ms)')

plt.savefig('export/' + machine + '/latency_per_message_rate_whisker.png', dpi=300, bbox_inches='tight', transparent = False)